# **Running Jupyter Code with Ollama Server**

1. **Download and Install Ollama**: Ensure that Ollama is downloaded and installed on your system.

2. **Check Ollama Version**:
   ```bash
   ollama --version
   ```

3. **Run Llama 3.2 Model**:
   To load the Llama 3.2 model:
   ```bash
   ollama run llama3.2
   ```

4. **Start Ollama Server**:
   To start the Ollama server, run:
   ```bash
   ollama serve
   ```

**Note**: If you face any issues while running the server, checking the background tasks using the port can help resolve conflicts. Always ensure the port is free before starting the server.

1. **Check if Ollama Server is Running**:
   Check if the server is running and using port 11434:
   ```bash
   netstat -ano | findstr :11434
   ```

2. **Identify and Kill Background Task (if needed)**:
   If you see that another task is using the port, you can identify and kill it:
   ```bash
   tasklist /FI "PID eq <PID>"
   taskkill /PID <PID> /F
   ```

3. **Close Ollama from Windows Task Bar (Optional)**:
   If the server is still not responding or you see issues, you can use the **up arrow key** (^) on the Windows task bar (bottom-left) to close any running Ollama instances.

4. **Start Jupyter Notebook**:
   Once the server is running, start your Jupyter notebook with the necessary code to interact with Ollama.

In [1]:
import ollama
from PyPDF2 import PdfReader

In [2]:
system_prompt = '''You are an agent which analyses the resume and understands the context of resume and 
    extract the following fields and format the output:
    - Name
    - Email
    - Phone 1
    - Phone 2
    - Address
    - City
    - LinkedIn
    - Professional Experience (in years)
    - Highest Education
    - Is Fresher (yes/no)
    - Is Student (yes/no)
    - Skills (comma-separated)
    - Applied For Profile
    - Education (Institute Name, Year of Passing, Score)
    - Projects (Project Name, Description)
    - Professional Experience (Organisation Name, Duration, Profile)

    Ensure to format the output as:
    Name: [Value]
    Email: [Value]
    Phone 1: [Value]
    ... and provide this summary of resume in markdown format'''

In [3]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, "rb") as file:
            reader = PdfReader(file)
            for page in reader.pages:
                text += page.extract_text()  # Extract text from each page
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
    return text
    
user_prompt = extract_text_from_pdf('./data/python-developer-resume-example.pdf') + "Provide a summary of this resume"

In [ ]:
import requests
import json

url = "http://localhost:11434/api/chat"
headers = {
    "Content-Type": "application/json"
}

# Define the conversation prompt and specify the model
data = {
    "model": "llama3.2",  # Specify the model name here
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
}

# Send the POST request to the Ollama server
response = requests.post(url, json=data, headers=headers, stream=True)

# Check if the response is successful
if response.status_code == 200:
    # Initialize a variable to accumulate the assistant's response
    full_response = ""
    
    # Iterate over the chunks of data as they arrive
    for chunk in response.iter_lines():
        if chunk:  # Ignore empty lines
            try:
                # Each chunk is a part of the message, so we can parse it as JSON
                chunk_data = json.loads(chunk.decode('utf-8'))
                
                # Extract the message content
                message_content = chunk_data.get('message', {}).get('content', '')
                full_response += message_content
                
                # Check if the response is done
                if chunk_data.get('done', False):
                    break
                
            except json.JSONDecodeError as e:
                print("Error parsing JSON chunk:", e)
    
    print("Model Response:", full_response)
else:
    print(f"Error {response.status_code}: {response.text}")


<Response [200]>

Model Response: ### Resume Summary

#### Contact Information
| Field | Value |
| --- | --- |
| Name | G I U L I A G O N Z A L E Z |
| Email | `gonzalez@email.com` |
| Phone 1 | `(123) 456-7890` |
| City | Detroit, MI |

#### Professional Experience
| Field | Value |
| --- | --- |
| Organisation Name | DoorDash |
| Duration | September 2017 - current |
| Profile | Python Developer |

### Education

| Field | Value |
| --- | --- |
| Institute Name | University of Chicago, University of Pittsburgh |
| Year of Passing | 2016, 2014 |
| Score | M.S. Computer Science (University of Chicago), B.S. Computer Science (University of Pittsburgh) |

#### Skills
`HTML/ CSS, SQL (PostgreSQL, Oracle), JavaScript (Angular), Python (Django), REST APIs (GraphQL), AWS (Redshift, S3), Git`

#### Applied For Profile
Not specified in the provided information.

### Projects

| Field | Value |
| --- | --- |
| Project Name | Cryptocurrency Price Tracker |
| Description | Creator |

#### Work Experience Details



## **OLD CODE** 

In [7]:
import requests

url = "http://localhost:11434/api/chat"
headers = {
    "Content-Type": "application/json"
}

# Define the conversation prompt and specify the model
data = {
    "model": "llama3.2",  # Specify the model name here
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ]
}

# Send the POST request to the Ollama server
response = requests.post(url, json=data, headers=headers)

# Print the raw response text for debugging
print("Raw Response:", response.text)

# Try to parse the response only if it's in JSON format
try:
    response_json = response.json()  # Try parsing the response as JSON
    print("Model Response:", response_json)
except ValueError as e:
    print("Error parsing JSON:", e)
    print("Response text may not be valid JSON.")


Raw Response: {"model":"llama3.2","created_at":"2024-12-18T09:48:16.4898854Z","message":{"role":"assistant","content":"The"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T09:48:16.5647504Z","message":{"role":"assistant","content":" capital"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T09:48:16.6424078Z","message":{"role":"assistant","content":" of"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T09:48:16.7355952Z","message":{"role":"assistant","content":" France"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T09:48:16.8292194Z","message":{"role":"assistant","content":" is"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T09:48:16.951255Z","message":{"role":"assistant","content":" Paris"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T09:48:17.0301931Z","message":{"role":"assistant","content":"."},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T09:48:17.1088063Z","message":{"role":"assistant","content